In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install gtts
!pip install playsound

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, GRU
from tensorflow.keras.activations import softmax, tanh, relu
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras import Input
from tensorflow.io import read_file
from tensorflow.image import decode_jpeg, resize
from tensorflow.data import Dataset
from tensorflow.data.experimental import AUTOTUNE

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.translate.bleu_score import sentence_bleu

from collections import Counter
import string
import time
from PIL import Image
from tqdm import tqdm
import glob
from gtts import gTTS
from playsound import playsound
from IPython import display
import collections
from wordcloud import WordCloud

# Creating dataframe

In [ ]:
images = '/kaggle/input/flickr8k/Images'

all_imgs = glob.glob(images + '/*.jpg',recursive=True)
print("The total images present in the dataset: {}".format(len(all_imgs)))

In [ ]:
txt_file = '/kaggle/input/flickr8k/captions.txt'

def load_doc(filename):
    open_file = open(txt_file, 'r', encoding='latin-1')
    txt = open_file.read() 
    open_file.close()
    
    return txt

doc = load_doc(txt_file)
print(doc[:500])

In [ ]:
imgs = '/kaggle/input/flickr8k/Images/'

img_id = [] # All image ID
img_path = [] # All image paths
captions = [] # All captions

with open('/kaggle/input/flickr8k/captions.txt' , 'r') as fo:
    next(fo) 
    for line in fo :
        split_arr = line.split(',')
        img_id.append(split_arr[0])
        captions.append(split_arr[1].rstrip('\n.')) 
        img_path.append(imgs + split_arr[0])

df = pd.DataFrame(list(zip(img_id, img_path, captions)),columns =['Image ID','Image Path','Captions']) 
    
df

# Visualizing Data

In [ ]:
vocab = [word.lower() for line in captions for word in line.split()]

### Bar Graph

In [ ]:
counter = Counter(vocab) 

for word, count in counter.most_common(10):
    print(word, ":", count)

lst = counter.most_common(10)
freq_df = pd.DataFrame(lst, columns = ['Word', 'Frequency'])
freq_df.plot.bar(x='Word', y='Frequency', width=0.6, color='blue', figsize=(15, 10))
plt.title("Top 10 maximum frequency words", fontsize = 18, color= 'navy')
plt.xlabel("Words", fontsize = 14, color= 'navy')
plt.ylabel("Frequency", fontsize = 14, color= 'navy')

### WordCloud

In [ ]:
wordcloud = WordCloud(width = 1000, height = 500).generate_from_frequencies(counter)
plt.figure(figsize = (12, 12))
plt.imshow(wordcloud)

### Image + Caption

In [ ]:
def plot_img_cap(img_id, df) :
    capt = ("\n" *2).join(df[df['Image ID'] == img_id]['Captions'].to_list())
    fig, ax = plt.subplots()
    ax.set_axis_off()
    idx = df['Image ID'].to_list().index(img_id)
    im = Image.open(df['Image Path'].iloc[idx])
    w, h = im.size[0], im.size[-1]
    ax.imshow(im)
    ax.text(w+50, h, capt, fontsize = 18, color = 'navy')

In [ ]:
for r in range(0,5) :
    plot_img_cap(df['Image ID'].drop_duplicates().iloc[r], df)

# Cleaning and Preprocessing Data

In [ ]:
rem_punct = str.maketrans('', '', string.punctuation)
for r in range(len(captions)) :
    line = captions[r]
    line = line.split()

    line = [word.lower() for word in line]
    line = [word.translate(rem_punct) for word in line]
    line = [word for word in line if len(word) > 1]
    line = [word for word in line if word.isalpha()]

    captions[r] = ' '.join(line)

In [ ]:
captions = ['<start>' + ' ' + line + ' ' + '<end>' for line in captions]

In [ ]:
captions[0:5]

In [ ]:
counter_2 = 5000
tokenizer = Tokenizer(num_words = counter_2+1, filters= '!"#$%^&*()_+.,:;-?/~`{}[]|\=@ ',
                      lower = True, char_level = False, 
                      oov_token = 'UNK')

In [ ]:
tokenizer.fit_on_texts(captions)

In [ ]:
seqs = tokenizer.texts_to_sequences(captions)

In [ ]:
tokenizer.word_index['PAD'] = 0
tokenizer.index_word[0] = 'PAD'   # Adding padding to words

In [ ]:
seqs_len = [len(seq) for seq in seqs]
longest = max(seqs_len)
cap_vector= pad_sequences(seqs, padding= 'post', maxlen = longest,
                                                          dtype='int32', value=0)

cap_vector.shape

In [ ]:
preprocessed = []
IMAGE_SHAPE = (255, 255)

In [ ]:
def load_images(image_path):
    img = read_file(image_path, name = None)
    img = decode_jpeg(img, channels=0)
    img = resize(img, IMAGE_SHAPE)
    img = preprocess_input(img)
    return img, image_path

In [ ]:
new_paths = sorted(set(img_path))
new_images = Dataset.from_tensor_slices(new_paths)
new_images = new_images.map(load_images, num_parallel_calls = AUTOTUNE)
new_images = new_images.batch(64, drop_remainder=False)

In [ ]:
new_images

In [ ]:
img_train, img_test, cap_train, cap_test = train_test_split(img_path, cap_vector, test_size = 0.2, random_state = 42)

In [ ]:
print("Training data for images: " + str(len(img_train)))
print("Testing data for images: " + str(len(img_test)))
print("Training data for Captions: " + str(len(cap_train)))
print("Testing data for Captions: " + str(len(cap_test)))

# Feature extraction

In [ ]:
img_model = tf.keras.applications.InceptionV3(include_top=False,weights='imagenet')

new_inp = img_model.input 
hidden = img_model.layers[-1].output 
extractor = tf.compat.v1.keras.Model(new_inp, hidden)

In [ ]:
extractor.summary()

In [ ]:
img_features = {}
for img, img_path in tqdm(new_images):
    
    batch_features = extractor(img)
    batch_features_flat = tf.reshape(batch_features, (batch_features.shape[0], -1, batch_features.shape[3]))

    for batch_feat, path in zip(batch_features_flat, img_path):
        feat_path = path.numpy().decode('utf-8')
        img_features[feat_path] = batch_feat.numpy()

In [ ]:
len(img_features)

In [ ]:
batch_feat.shape

In [ ]:
def map(img_name, cap):
    img_tensor = img_features[img_name.decode('utf-8')]
    return img_tensor, cap

In [ ]:
BUFFER_SIZE = 1000
BATCH_SIZE = 64
def gen_ds(img, cap):
    
    data = Dataset.from_tensor_slices((img, cap))
    data = data.map(lambda ele1, ele2 : tf.numpy_function(map, [ele1, ele2], [tf.float32, tf.int32]),
                    num_parallel_calls = AUTOTUNE)
    
    data = (data.shuffle(BUFFER_SIZE, reshuffle_each_iteration= True).batch(BATCH_SIZE, drop_remainder = False)
    .prefetch(AUTOTUNE))

    return data

In [ ]:
train_ds = gen_ds(img_train, cap_train)
test_ds = gen_ds(img_test,cap_test)

In [ ]:
sample_img_batch, sample_cap_batch = next(iter(train_ds))
print(sample_img_batch.shape)  
print(sample_cap_batch.shape) 

# Model

In [ ]:
embedding_dim = 256 
units = 512

vocab_size = 5001
train_steps = len(img_train) // BATCH_SIZE 
test_steps = len(img_test) // BATCH_SIZE  

max_length = 31
feat_shape = batch_feat.shape[1]
attn_feat_shape = batch_feat.shape[0]

### Encoder

In [ ]:
class Encoder(Model):
    def __init__(self,embed_dim):
        super(Encoder, self).__init__()
        self.dense = Dense(embed_dim) 
        
    def call(self, features):
        features =  self.dense(features) 
        features =  relu(features, alpha=0.01, max_value=None, threshold=0)
        return features

In [ ]:
encoder=Encoder(embedding_dim)

### Attention Model

In [ ]:
class Attn_model(Model):
    def __init__(self, units):
        super(Attn_model, self).__init__()
        self.W1 = Dense(units) 
        self.W2 = Dense(units)
        self.V = Dense(1) 
        self.units=units

    def call(self, features, hidden):
        
        hidden_time = hidden[:, tf.newaxis]
        score = tanh(self.W1(features) + self.W2(hidden_time))  
        attn_weights = softmax(self.V(score), axis=1) 
        context_vector = attn_weights * features 
        context_vector = tf.reduce_sum(context_vector, axis=1)  
        
        return context_vector, attn_weights

### Decoder

In [ ]:
class Decoder(Model):
    def __init__(self, embed_dim, units, vocab_size):
        super(Decoder, self).__init__()
        self.units = units
        self.attention = Attn_model(self.units) 
        self.embed = Embedding(vocab_size, embed_dim) 
        self.gru = GRU(self.units,return_sequences=True, return_state=True,recurrent_initializer='glorot_uniform')
        self.d1 = Dense(self.units) 
        self.d2 = Dense(vocab_size) 
        

    def call(self,x,features, hidden):
        context_vector, attn_weights = self.attention(features, hidden) 
        embed = self.embed(x) 
        embed = tf.concat([tf.expand_dims(context_vector, 1), embed], axis = -1)  
        output,state = self.gru(embed) 
        output = self.d1(output)
        output = tf.reshape(output, (-1, output.shape[2])) 
        output = self.d2(output)
        
        return output, state, attn_weights
    
    def init_state(self, batch_size):
        return tf.zeros((batch_size, self.units))

In [ ]:
decoder=Decoder(embedding_dim, units, vocab_size)

In [ ]:
features=encoder(sample_img_batch)

hidden = decoder.init_state(batch_size=sample_cap_batch.shape[0])
dec_in = tf.expand_dims([tokenizer.word_index['<start>']] * sample_cap_batch.shape[0], 1)

pred, hidden_out, attn_weights= decoder(dec_in, features, hidden)
print('Feature shape from Encoder: {}'.format(features.shape)) #(batch, 8*8, embed_dim)
print('Predictions shape from Decoder: {}'.format(pred.shape)) #(batch,vocab_size)
print('Attention weights shape from Decoder: {}'.format(attn_weights.shape)) #(batch, 8*8, embed_dim)

# Model training

In [ ]:
optimizer = Adam(learning_rate = 0.001)  
loss_object = SparseCategoricalCrossentropy(from_logits = True, reduction = tf.keras.losses.Reduction.NONE) 

In [ ]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    
    return tf.reduce_mean(loss_)

In [ ]:
checkpoint_path = "Flickr8K/checkpoint1"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [ ]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])

In [ ]:
@tf.function
def train_step(img_tensor, target):
    loss = 0
    hidden = decoder.init_state(batch_size=target.shape[0])
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)
    
    with tf.GradientTape() as tape:

        encoder_op = encoder(img_tensor)

        for r in range(1, target.shape[1]) :
            predictions, hidden, _ = decoder(dec_input, encoder_op, hidden)
            loss = loss + loss_function(target[:, r], predictions) 
            dec_input = tf.expand_dims(target[:, r], 1)  

    avg_loss = (loss/int(target.shape[1])) 
    trainable_vars = encoder.trainable_variables + decoder.trainable_variables
    grad = tape.gradient(loss, trainable_vars) 
    optimizer.apply_gradients(zip(grad, trainable_vars))

    return loss, avg_loss

In [ ]:
@tf.function
def test_step(img_tensor, target):
    loss = 0
    
    hidden = decoder.init_state(batch_size = target.shape[0])

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)
    with tf.GradientTape() as tape:
        encoder_op = encoder(img_tensor)

        for r in range(1, target.shape[1]) :
            predictions, hidden, _ = decoder(dec_input, encoder_op, hidden)
            loss = loss + loss_function(target[:, r], predictions)

            dec_input = tf.expand_dims(target[: , r], 1)

    avg_loss = (loss/ int(target.shape[1]))
    trainable_vars = encoder.trainable_variables + decoder.trainable_variables
    grad = tape.gradient (loss, trainable_vars)
    
    optimizer.apply_gradients(zip(grad, trainable_vars))                      

    return loss, avg_loss

In [ ]:
def test_loss_cal(test_dataset):
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(test_dataset) :
        batch_loss, t_loss = test_step(img_tensor, target)
        total_loss = total_loss + t_loss
        avg_test_loss = total_loss/ test_steps

    return avg_test_loss

In [ ]:
loss_plot = []
test_loss_plot = []
EPOCHS = 15

best_test_loss=100

for epoch in tqdm(range(0, EPOCHS)):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(train_ds):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss
        avg_train_loss = total_loss / train_steps
        
    loss_plot.append(avg_train_loss)    
    test_loss = test_loss_cal(test_ds)
    test_loss_plot.append(test_loss)
    
    print ('For epoch: {}, the train loss is {:.3f}, & test loss is {:.3f}'.format(epoch+1,avg_train_loss,test_loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
    
    if test_loss < best_test_loss:
        print('Test loss has been reduced from %.3f to %.3f' % (best_test_loss, test_loss))
        best_test_loss = test_loss
        ckpt_manager.save()

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(loss_plot, color='orange', label = 'training_loss_plot')
plt.plot(test_loss_plot, color='green', label = 'test_loss_plot')
plt.xlabel('Epochs', fontsize = 15, color = 'red')
plt.ylabel('Loss', fontsize = 15, color = 'red')
plt.title('Loss Plot', fontsize = 20, color = 'red')
plt.legend()
plt.show()

# Model Evaluation

In [ ]:
def evaluate(image):
    attn_plot = np.zeros((max_length, attn_feat_shape))

    hidden = decoder.init_state(batch_size=1)

    temp_inp = tf.expand_dims(load_images(image)[0], 0) 
    img_tensor_val = extractor(temp_inp) 
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val) 

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length):
        pred, hidden, attn_weights = decoder(dec_input, features, hidden) 

        attn_plot[i] = tf.reshape(attn_weights, (-1, )).numpy()

        pred_id = tf.argmax(pred[0]).numpy() 
        result.append (tokenizer.index_word[pred_id])

        if tokenizer.index_word[pred_id] == '<end>':
            return result, attn_plot, pred

        dec_input = tf.expand_dims([pred_id], 0)

    attn_plot = attn_plot[:len(result), :]
    return result, attention_plot, pred

In [ ]:
def plot_attn_map (caption, weights, img) :

    fig = plt.figure(figsize = (10, 10))
    temp_img = np.array(Image.open(img))

    cap_len = len(caption)
    for cap in range(cap_len) :
        weights_img_1 = np.reshape(weights[cap], (6, 6))
        weights_img_2 = np.array(Image.fromarray(weights_img_1).resize((224,224), Image.LANCZOS))

        ax = fig.add_subplot(cap_len//2, cap_len//2, cap+1)
        ax.set_title(caption[cap], fontsize = 14, color = 'red')

        img = ax.imshow(temp_img)

        ax.imshow(weights_img_1, cmap='gist_heat', alpha=0.6, extent=img.get_extent())
        ax.axis('off')
    plt.subplots_adjust(hspace=0.2, wspace=0.2)
    plt.show()

In [ ]:
def filt_text(text):
    filt=['<start>','<unk>','<end>'] 
    temp= text.split()
    [temp.remove(j) for k in filt for j in temp if k==j]
    text=' '.join(temp)
    return text

In [ ]:
image_test = img_test.copy()

In [ ]:
def pred_caption_audio(random, autoplay=False, weights=(0.5, 0.5, 0, 0)) :

    cap_test_data = cap_test.copy()
    rid = np.random.randint(0, random)
    test_img = image_test[rid]

    real_caption = ' '.join([tokenizer.index_word[i] for i in cap_test_data[rid] if i not in [0]])
    result, attention_plot, pred_test = evaluate(test_img)

    real_caption=filt_text(real_caption)      

    pred_caption=' '.join(result).rsplit(' ', 1)[0]

    real_appn = []
    real_appn.append(real_caption.split())
    reference = real_appn
    candidate = pred_caption.split()

    score = sentence_bleu(reference, candidate, weights=weights)#set your weights
    print(f"BLEU score: {score*100}")
    print ('Real Caption:', real_caption)
    print ('Prediction Caption:', pred_caption)
    plot_attn_map(result, attention_plot, test_img)
    
    speech = gTTS(pred_caption, lang = 'en', slow = False)
    speech.save('voice.mp3')
    audio_file = 'voice.mp3'

    display.display(display.Audio(audio_file, rate = None, autoplay = autoplay))

    return test_img

### Img 1

In [ ]:
test_image = pred_caption_audio(len(image_test), True, weights = (0.5, 0.25, 0, 0))
Image.open(test_image)

### Img 2

In [ ]:
test_image = pred_caption_audio(len(image_test), True, weights = (0.5, 0.25, 0, 0))
Image.open(test_image)

### Img 3

In [ ]:
test_image = pred_caption_audio(len(image_test), True, weights = (0.5, 0.25, 0, 0))
Image.open(test_image)

### Img 4

In [ ]:
test_image = pred_caption_audio(len(image_test), True, weights = (0.35, 0.25, 0, 0))
Image.open(test_image)

### Img 5

In [ ]:
test_image = pred_caption_audio(len(image_test), True, weights = (0.5, 0.5, 0, 0))
Image.open(test_image)